In [1]:
print("Hello World!")

Hello World!


In [10]:
import cv2
import numpy as np

# Inicializar la captura de video desde la webcam (índice 0)
cap = cv2.VideoCapture(0)

# Definir el kernel
kernel = np.ones((5, 5), np.uint8)

# Crear una ventana de visualización
cv2.namedWindow('Estilizado en Tiempo Real', cv2.WINDOW_NORMAL)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    smoothed = cv2.bilateralFilter(frame, d=9, sigmaColor=75, sigmaSpace=75)

    # Redimensionar el marco para procesamiento más rápido
    frame = cv2.resize(frame, (640, 480))

    # Convertir a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Aplicar cierre
    closed = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)

    # Aplicar apertura
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel)

    # Aplicar binarización
    _, binary = cv2.threshold(opened, 120, 255, cv2.THRESH_BINARY_INV)

    # Calcular el gradiente morfológico
    gradient = cv2.morphologyEx(opened, cv2.MORPH_GRADIENT, kernel)

    # Invertir el gradiente para obtener bordes en blanco sobre fondo negro
    gradient_inv = cv2.bitwise_not(gradient)

    # Convertir el gradiente a tres canales
    gradient_colored = cv2.cvtColor(gradient_inv, cv2.COLOR_GRAY2BGR)

    # Aplicar el efecto de dibujo
    styled_frame = cv2.bitwise_and(smoothed, gradient_colored)

    # Mostrar el fotograma en proceso
    cv2.imshow('Estilizado en Tiempo Real', styled_frame)

    # Romper el ciclo con 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()


## Ejemplos con imagenes

In [2]:
import cv2
import os
import numpy as np

# Obtener la ruta de la carpeta actual
current_folder = os.getcwd()
imgs_folder = os.path.join(current_folder, 'imgs')

# Función para aplicar un efecto de dibujo animado usando operaciones morfológicas
def cartoonize_image_morph(image_path):
    # Leer la imagen
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: no se puede leer la imagen {image_path}")
        return

    # Convertir a escala de grises
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Aplicar la operación de gradiente morfológico
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    gradient = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, kernel)

    # Aplicar un umbral al gradiente
    _, gradient_thresh = cv2.threshold(gradient, 40, 220, cv2.THRESH_BINARY)

    # Invertir el gradiente para obtener los bordes en blanco
    gradient_inv = cv2.bitwise_not(gradient_thresh)
    
    # Expande el gradiente invertido a tres canales para que coincida con la imagen original
    expanded_gradient_inv = cv2.merge([gradient_inv, gradient_inv, gradient_inv])

    # Combinar la imagen original con los bordes
    cartoon = cv2.bitwise_and(img, expanded_gradient_inv)

    return cartoon

# Procesar todas las imágenes en la carpeta 'imgs'
for filename in os.listdir(imgs_folder):
    # Filtrar solo archivos de imagen comunes
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.webp')):
        input_path = os.path.join(imgs_folder, filename)
        output_path_cartoon = os.path.join(current_folder, f'cartoon_{filename}')
        
        # Procesar la imagen
        cartoon_image = cartoonize_image_morph(input_path)
        
        # Guardar la imagen procesada
        if cartoon_image is not None:
            cv2.imwrite(output_path_cartoon, cartoon_image)

print("Procesamiento completado. Las imágenes se han guardado en la carpeta actual.")


Procesamiento completado. Las imágenes se han guardado en la carpeta actual.
